In [8]:
import csv
from collections import defaultdict

class NFAToDFAConverter:
    def __init__(self, filename: str):
        self.filename = filename
        self.nfa_transitions = defaultdict(list)
        self.start_state = None
        self.accept_states = set()

    def epsilon_closure(self, states):
        stack = list(states)
        closure = set(states)

        while stack:
            state = stack.pop()
            for next_state in self.nfa_transitions.get((state, ''), []):
                if next_state not in closure:
                    closure.add(next_state)
                    stack.append(next_state)
        return closure

    def nfa_to_dfa(self):
        dfa = defaultdict(list)
        dfa_start_state = '+'.join(sorted(self.epsilon_closure([self.start_state])))
        dfa_accept_states = set()
        unmarked_states = [dfa_start_state]
        dfa_states = {dfa_start_state}

        while unmarked_states:
            current_dfa_state = unmarked_states.pop()
            nfa_states = set(current_dfa_state.split('+'))

            # Process for each symbol in the NFA transitions excluding epsilon
            for symbol in set(sym for state, sym in self.nfa_transitions if sym):
                next_states = set()
                for state in nfa_states:
                    for next_state in self.nfa_transitions.get((state, symbol), []):
                        next_states |= self.epsilon_closure([next_state])

                if next_states:
                    next_dfa_state = '+'.join(sorted(next_states))
                    dfa[(current_dfa_state, symbol)] = [next_dfa_state]

                    if next_dfa_state not in dfa_states:
                        dfa_states.add(next_dfa_state)
                        unmarked_states.append(next_dfa_state)

        # Determine DFA accept states
        for dfa_state in dfa_states:
            if any(state in self.accept_states for state in dfa_state.split('+')):
                dfa_accept_states.add('*' + dfa_state)

        return dfa, dfa_start_state, dfa_accept_states

    def read_nfa_from_file(self):
        try:
            with open(self.filename) as file:
                reader = csv.reader(file, delimiter=',')
                for line_number, line in enumerate(reader):
                    if line_number == 3:
                        self.start_state = line[0]
                    elif line_number > 4:
                        state, symbol, next_state = line[0], line[1] or '', line[2]
                        self.nfa_transitions[(state, symbol)].append(next_state)
                        if symbol == '':
                            self.accept_states.add(next_state)
        except FileNotFoundError:
            print(f"Error: The file '{self.filename}' was not found.")
            exit(1)

    def write_dfa_to_file(self, dfa, dfa_start_state, dfa_accept_states):
        output_filename = f"DFA_{self.filename}"
        with open(output_filename, 'w', newline='') as file:
            writer = csv.writer(file)

            # Writing the header
            writer.writerow(['DFA,,,'])  # Replace 'N1' with actual DFA name if available
            states = sorted({state for state, _ in dfa} | {next_state for _, next_states in dfa.items() for next_state in next_states})
            writer.writerow(states + [''] * (4 - len(states)))
            alphabet = sorted({symbol for _, symbol in dfa})
            writer.writerow(alphabet + [''] * (3 - len(alphabet)))
            writer.writerow([dfa_start_state] + [''] * 3)
            writer.writerow(sorted(dfa_accept_states) + [''] * (4 - len(dfa_accept_states)))

            # Writing transitions
            for (state, symbol), next_states in dfa.items():
                for next_state in next_states:
                    writer.writerow([state, symbol, next_state, ''])

        print(f"DFA saved to {output_filename}")

    def convert_and_export(self):
        self.read_nfa_from_file()
        dfa, dfa_start_state, dfa_accept_states = self.nfa_to_dfa()
        self.write_dfa_to_file(dfa, dfa_start_state, dfa_accept_states)

def main():
    filename = input("Enter the filename of the NFA: ")
    converter = NFAToDFAConverter(filename)
    converter.convert_and_export()

if __name__ == "__main__":
    main()


DFA saved to DFA_N1.csv


In [ ]:
import copy
import json

class FSA:
    def __init__(self, states, alphabet, transitions, start_state, accept_states):
        self.states = states
        self.alphabet = alphabet
        self.transitions = transitions
        self.start_state = start_state
        self.accept_states = accept_states

    def clone(self):
        return FSA(
            json.loads(json.dumps(self.states)),
            json.loads(json.dumps(self.alphabet)),
            json.loads(json.dumps(self.transitions)),
            self.start_state,
            json.loads(json.dumps(self.accept_states))
        )

    def remove_state(self, state):
        self.states = [s for s in self.states if s != state]
        self.accept_states = [s for s in self.accept_states if s != state]
        if self.start_state == state:
            self.start_state = None
        self.transitions.pop(state, None)

        for from_state, transitions in self.transitions.items():
            for symbol, target_states in list(transitions.items()):
                if state in target_states:
                    target_states.remove(state)
                if not target_states:
                    self.transitions[from_state].pop(symbol, None)

    def merge_states(self, s1, s2):
        new_state = f"{s1}+{s2}"
        self.states.append(new_state)
        if s1 in self.accept_states:
            self.accept_states.append(new_state)
        if self.start_state == s1 or self.start_state == s2:
            self.start_state = new_state

        self.transitions[new_state] = {symbol: [new_state] for symbol in self.alphabet}
        
        for state in [s for s in self.states if s != s1 and s != s2]:
            for symbol in self.alphabet:
                if self.transitions[state][symbol][0] in [s1, s2]:
                    self.transitions[state][symbol] = [new_state]

        self.remove_state(s1)
        self.remove_state(s2)

    def get_powerset_of_states(self):
        result = [['Ø']]

        for i in range(1, 1 << len(self.states)):
            subset = [self.states[j] for j in range(len(self.states)) if i & (1 << j)]
            result.append(sorted(subset))

        return result

    def get_epsilon_closure_states(self, from_state):
        if from_state not in self.states:
            raise ValueError(f"Unknown state: {from_state}")

        if 'ε' not in self.transitions.get(from_state, {}):
            return [from_state]
        else:
            return [from_state, *self.transitions[from_state]['ε']]

    def get_reachable_states(self, from_state, symbol, list=None):
        if list is None:
            list = []

        if from_state not in self.states:
            raise ValueError(f"Unknown state: {from_state}")
        if symbol != 'ε' and symbol not in self.alphabet:
            raise ValueError(f"Unknown symbol: {symbol}")
        if len(list) > 200:
            raise ValueError("Infinite transition loop detected in NFA")

        if symbol not in self.transitions.get(from_state, {}):
            return ['Ø'] if symbol != 'ε' else []

        list += self.transitions[from_state][symbol]

        for s in self.transitions[from_state][symbol]:
            if s in self.transitions and 'ε' in self.transitions[s]:
                list = self.get_reachable_states(s, 'ε', list)

        return sorted(set(list))


In [ ]:
class NFAConverter:
    def __init__(self, filename):
        self.filename = filename
        self.nfa = None 
        self.dfa = None
        self.steps = []
        self.state_index = 0
        self.alphabet_index = 0
        self.unreachable_states = None
        self.redundant_states = None

    def get_next_step(self):
        if self.dfa is None:
            return 'initialize'
        if self.state_index < len(self.dfa.states):
            return 'add_transition'

        if self.unreachable_states is None:
            self.unreachable_states = self.get_unreachable_states()
        if self.unreachable_states:
            return 'delete_state'

        if self.redundant_states is None:
            self.redundant_states = self.get_redundant_states()
        if self.redundant_states:
            return 'merge_states'

    def get_unreachable_states(self, temp_dfa=None, list=None):
        if list is None:
            list = []

        if temp_dfa is None:
            temp_dfa = self.dfa.clone()

        nodes_with_incoming_edges = []

        for state in temp_dfa.states:
            for symbol in temp_dfa.alphabet:
                node = ','.join(temp_dfa.transitions[state][symbol])
                if node != state:
                    nodes_with_incoming_edges.append(node)

        nodes_without_incoming_edges = [
            s for s in temp_dfa.states 
            if s not in nodes_with_incoming_edges and s != temp_dfa.start_state
        ]

        if nodes_without_incoming_edges:
            for n in nodes_without_incoming_edges:
                temp_dfa.remove_state(n)
            list += self.get_unreachable_states(temp_dfa, list + nodes_without_incoming_edges)

        return sorted(set(list))

    def get_redundant_states(self, temp_dfa=None, list=None):
        if list is None:
            list = []

        if temp_dfa is None:
            temp_dfa = self.dfa.clone()

        for s1 in temp_dfa.states:
            for s2 in [s for s in temp_dfa.states if s != s1]:
                if ((s1 in temp_dfa.accept_states and s2 in temp_dfa.accept_states) or
                    (s1 not in temp_dfa.accept_states and s2 not in temp_dfa.accept_states)):
                    redundant = True
                    for symbol in temp_dfa.alphabet:
                        t1 = temp_dfa.transitions[s1][symbol][0]
                        t2 = temp_dfa.transitions[s2][symbol][0]
                        if t1 not in [s1, s2] or t2 not in [s2, s1]:
                            redundant = False
                    if redundant:
                        temp_dfa.merge_states(s1, s2)
                        list.append([s1, s2])
                        return self.get_redundant_states(temp_dfa, list)

        return list

    def initialize_dfa(self):
        powerset = self.nfa.get_powerset_of_states()
        states = [','.join(e) for e in powerset]
        transitions = {s: {e: None for e in self.nfa.alphabet} for s in states}

        start_state = ','.join(sorted(set(self.nfa.get_epsilon_closure_states(self.nfa.start_state))))

        accept_states = [
            ','.join(e) for e in powerset if any(s in self.nfa.accept_states for s in e)
        ]

        if start_state not in states:
            raise ValueError(f"startState {start_state} not in states")

        self.dfa = FSA(states, self.nfa.alphabet, transitions, start_state, accept_states)

    def add_transition(self):
        current_state = self.dfa.states[self.state_index]
        current_symbol = self.nfa.alphabet[self.alphabet_index]

        eps_closure_states = [
            self.nfa.get_epsilon_closure_states(state) for state in current_state.split(',')
        ]

        reachable_states = [
            self.nfa.get_reachable_states(state, current_symbol) for state in itertools.chain(*eps_closure_states)
        ]

        destination_state = ','.join(sorted(set(itertools.chain(*reachable_states))))

        self.dfa.transitions[current_state][current_symbol] = [destination_state]

        self.alphabet_index += 1

        if self.alphabet_index >= len(self.nfa.alphabet):
            self.state_index += 1
            self.alphabet_index = 0
    def read_nfa_from_csv(self):
        with open(self.filename, newline='') as file:
            reader = csv.reader(file)
            lines = list(reader)
        
        # Extract states, alphabet, start state, and accept states
        states = lines[1]
        alphabet = lines[0][1:]
        transitions = {state: {a: [] for a in alphabet} for state in states}
        start_state = None
        accept_states = []

        for state in states:
            if state.startswith("*"):
                accept_states.append(state.strip("*"))
                state = state.strip("*")
            if state.startswith("q1"):  # Assuming q1 is always the start state
                start_state = state

            for i, sym in enumerate(alphabet):
                if lines[states.index(state) + 2][i + 1]:
                    transitions[state][sym] = lines[states.index(state) + 2][i + 1].split(',')

        # Initialize the NFA
        self.nfa = FSA(states, alphabet, transitions, start_state, accept_states)

    def output_nfa(self):
        output = ["N1,,,"]  # NFA identifier
        output.append(",".join(self.nfa.states))
        output.append(",".join(["0"] + self.nfa.alphabet))
        for state in self.nfa.states:
            line = [state] + [",".join(self.nfa.transitions[state].get(a, [])) for a in self.nfa.alphabet]
            output.append(",".join(line))

        output_str = "\n".join(output)
        return output_str